# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계4 : 통합 - pipeline**

## **0.미션**

단계 4에서는, 단계1,2,3 에서 생성한 함수들을 모듈화하고, 단위 테스트 및 파이프라인 코드를 작성합니다.

* **미션6**
    * 단위 테스트
        * 각 기능(함수)에 대해 단계별로 테스트를 수행하며 오류를 해결합니다.
    * 파이프라인 구축
        * 단계1의 결과가 단계2 모델에 input이 되고, 모델의 예측 결과를 기반으로
        * 응급실 추천되도록
        * 조원들이 녹음한 음성 파일에 임의의 좌표(위도, 경도)값을 부여
            * 음성파일 이름과 좌표를 저장하는 별도 데이터셋 생성
        * 각 모듈을 연결하여 파이프라인 구성하는 ipynb 파일 생성



## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = '/content/drive/MyDrive/project6_2/'

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [3]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/project6_2/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#### 2) 라이브러리 로딩

In [52]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import sys
sys.path.append(path)

from transformers import AutoTokenizer, AutoModelForSequenceClassification

import sys
sys.path.append('/content/drive/MyDrive/project6_2')

# 조에서 생성한 모듈 불러오기 -------------/
import our_module_4_fin as emergency

## **2. 단위 테스트**

* 세부사항 : 아래 단계별로 데이터가 순차적으로 처리되도록 단위 테스트를 진행합니다.



하나씩 테스트해보고 막히는 데 있는지 확인~~

### (1) open ai key 등록

In [ ]:
# def set_openai_api_key(path):
#     filepath = path + 'api_key.txt'

#     try:
#         with open(filepath, 'r') as file:
#             api_key = file.readline().strip()
#         os.environ['OPENAI_API_KEY'] = api_key
#         print("OpenAI API 키가 설정되었습니다.")
#     except FileNotFoundError:
#         print(f"오류: 파일을 찾을 수 없습니다: {filepath}")
#     except Exception as e:
#         print(f"예기치 않은 오류가 발생했습니다: {e}")

def set_openai_api_key(filepath):
    filepath = filepath + 'api_key.txt'
    with open(filepath, 'r') as file:
        api_key = file.readline().strip()
    os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
set_openai_api_key(path)

### (2) audio to text

In [ ]:
def audio_to_text(audio_path, filename):
    audio_path = audio_path + 'audio/'
    client = OpenAI()

    with open(audio_path + filename, "rb") as audio_file:
        transcript = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            language="ko",
            response_format="text",
        )
    return transcript.strip()

In [ ]:
audio_to_text(path, 'audio2.mp3')

'119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 아프고 좀 띵한 것 같아요. 우한이 좀 들어요. 어떻게 해야 할까요?'

### (3) text summary

In [ ]:
def summarize_text(input_text):

    # OpenAI 클라이언트 생성
    client = OpenAI()

    # 시스템 역할과 응답 형식 지정
    system_role = '''당신은 119 응급대원에게 상황을 전달하기 위해 대화의 핵심을 키워드로 요약해주는 어시스턴트입니다.
    응답은 다음의 형식을 지켜주세요.
    {"summary": \"키워드\"}
    '''

    # GPT-3.5-turbo에 데이터 전달
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": system_role
            },
            {
                "role": "user",
                "content": input_text
            }
        ]
    )

    # 응답 데이터 파싱
    answer = response.choices[0].message.content
    parsed_answer = json.loads(answer)

    # 요약된 텍스트 반환
    return parsed_answer["summary"]

In [ ]:
summarize_text(audio_to_text(path, 'audio2.mp3'))

'열이 38도, 머리 아프고 띵함, 목이 아프고 우한함'

### (4) 응급실 등급분류

In [ ]:
import shutil
import zipfile
import os
# zip파일 불러오기
shutil.copy(f'{path}/fine_tuned_bert.zip', '/content/fine_tuned_bert.zip')
# 필요한 폴더 생성
os.makedirs('/content/fine_tuned_bert', exist_ok=True)
# 압축해제
with zipfile.ZipFile('/content/fine_tuned_bert.zip', 'r') as zip_my:
    zip_my.extractall('/content/fine_tuned_bert/')

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def model_prediction(text, model_directory):
    model = AutoModelForSequenceClassification.from_pretrained(model_directory).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_directory)

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probabilities = logits.softmax(dim=1)
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    grade = predicted_class + 1

    return grade


In [ ]:
model_prediction(summarize_text(audio_to_text(path, 'audio2.mp3')), '/content/fine_tuned_bert/')

3

In [ ]:
# def model_prediction(text, model_directory):
#   # 모델 로드
#   model = AutoModelForSequenceClassification.from_pretrained(model_directory)
#   # 토크나이저 로드
#   tokenizer = AutoTokenizer.from_pretrained(model_directory)
#   # 예측 수행
#   predicted_class, probabilities = predict(text, model, tokenizer)
#   # 등급 계산
#   grade = predicted_class + 1
#   return grade

### (5) 응급실추천

In [ ]:
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json

# 3-1. get_distance-----------------
class EmergencyRoomLocator:
    """
    네이버 지도 API를 사용하여 가장 가까운 응급실을 검색하는 클래스입니다!
    """

    def __init__(self, client_id, client_secret, csv_path, initial_radius=10, radius_increment=10, unit='km'):
        """
        클래스 초기화와 로드부분

        Parameters:
            client_id (str): 클라이언트 아이디
            client_secret (str): 클라이언트 비밀번호
            csv_path (str): 응급실 csv경로
        """
        self.client_id = client_id
        self.client_secret = client_secret
        self.initial_radius = initial_radius
        self.radius_increment = radius_increment
        self.unit = unit

        # 응급실 데이터 로드
        self.emergency_data = pd.read_csv(csv_path)

    def get_distance_from_naver_api(self, start_lat, start_lng, dest_lat, dest_lng):
        """
        네이버 지도 API를 사용하여 두 좌표 간의 도로 거리(km)를 계산합니다.

        Parameters:
            start_lat (float): 출발지의 위도
            start_lng (float): 출발지의 경도
            dest_lat (float): 목적지의 위도
            dest_lng (float): 목적지의 경도

        Returns:
            float: 도로 거리
        """
        url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
        headers = {
            "X-NCP-APIGW-API-KEY-ID": self.client_id,
            "X-NCP-APIGW-API-KEY": self.client_secret,
        }
        params = {
            "start": f"{start_lng},{start_lat}",
            "goal": f"{dest_lng},{dest_lat}",
            "option": "trafast"
        }
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            try:
                distance = data['route']['trafast'][0]['summary']['distance'] / 1000
                return round(distance, 2)
            except KeyError:
                return float('inf')
        else:
            raise ConnectionError(f"API 호출 실패: {response.status_code}, {response.text}")

# 3-2. recommendation------------------

    def find_nearest_emergency_rooms(self, input_coords, top_n=3):
        """
        입력된 좌표를 기준으로 도로 거리 기반 가장 가까운 응급실을 검색합니다.

        Parameters:
            input_coords (tuple): 입력 좌표 (위도 / 경도).
            top_n (int): 반환할 응급실 개수.

        Returns:
            pd.DataFrame: 가장 가까운 응급실 정보와 거리.
        """
        radius = self.initial_radius
        lat, lon = input_coords

        while True:
            R_km = 6371
            delta_lat = radius / R_km
            delta_lon = radius / (R_km * cos(radians(lat)))

            filtered_data = self.emergency_data[
                (self.emergency_data['위도'] >= lat - delta_lat) &
                (self.emergency_data['위도'] <= lat + delta_lat) &
                (self.emergency_data['경도'] >= lon - delta_lon) &
                (self.emergency_data['경도'] <= lon + delta_lon)
            ].copy()

            if not filtered_data.empty:
                filtered_data['Distance'] = filtered_data.apply(
                    lambda row: self.get_distance_from_naver_api(
                        lat, lon, row['위도'], row['경도']
                    ),
                    axis=1
                )
                filtered_data = filtered_data[filtered_data['Distance'] < float('inf')]
                nearest_emergency_rooms = filtered_data.sort_values(by='Distance').head(top_n)
                if len(nearest_emergency_rooms) >= top_n:
                    return nearest_emergency_rooms

            radius += self.radius_increment

## **3. 파이프라인**

* 세부사항
    * [2. 단계별 테스트] 의 내용을 순차적으로 정리합니다.
        * 데이터 처리 전 준비작업 : 한번 실행하면 되는 영역
            * 키, 데이터로딩
            * 모델/토크나이저 로딩
        * 입력값이 들어 왔을 때 출력값까지 처리되는 영역


즉 총정리!!

In [57]:
class EmergencyPipeline:
    def __init__(self, path, csv_path, model_directory):
        """
        파이프라인 준비 작업 포함 초기화.

        Parameters:
            path (str): API 키와 모델 관련 파일 경로.
            csv_path (str): 응급실 데이터 CSV 파일 경로.
            model_directory (str): 사전 학습된 모델 디렉토리 경로.
        """
        # 키 로딩
        keys = self.load_api_keys(path + 'map_key.txt')
        client_id = keys.get('c_id')
        client_secret = keys.get('c_key')

        # SpeechToTextConverter 초기화
        api_key = emergency.set_openai_api_key(path)
        self.converter = emergency.SpeechToTextConverter(api_key=api_key)

        # ModelPredictor 초기화
        self.predictor = emergency.ModelPredictor(model_directory)

        # EmergencyRoomLocator 초기화
        self.locator = emergency.EmergencyRoomLocator(client_id, client_secret, csv_path)

    @staticmethod
    def load_api_keys(file_path):
        """
        API 키 로드 함수.
        """
        try:
            with open(file_path, 'r') as file:
                keys = json.load(file)
            return keys
        except Exception as e:
            raise ValueError(f"API 키 파일 로드 중 오류 발생: {e}")

    def run(self, audio_path, filename, input_coords, top_k=3):
        """
        파이프라인 실행.
        """
        print("[Step 1] 오디오를 텍스트로 변환 중...")
        transcript = self.converter.audio_to_text(audio_path, filename)
        if not transcript:
            raise ValueError("오디오를 텍스트로 변환하는 데 실패했습니다.")
        print(f"Transcript: {transcript}")

        print("[Step 2] 텍스트를 요약 중...")
        summary = self.converter.summarize_text(transcript)
        if not summary:
            raise ValueError("텍스트 요약에 실패했습니다.")
        print(f"Summary: {summary}")

        print("[Step 3] 텍스트 등급 예측 중...")
        grade = self.predictor.predict(summary)
        print(f"Predicted Grade: {grade}")
        if grade >= 1:
          print("집에서 요양하면 충분할듯합니다.")

        else:
          print("[Step 4] 가장 가까운 응급실 검색 중...")
          nearest_emergency_rooms = self.locator.find_nearest_emergency_rooms(input_coords, top_k)
          print("Recommended Emergency Rooms:")

          return nearest_emergency_rooms

In [58]:
import os

def run_emergency_pipeline(audio_path, latitude, longitude, top_k=3):
    """
    응급실 추천 파이프라인 실행 함수.

    Parameters:
        audio_path (str): 오디오 파일 경로.
        latitude (float): 입력 위도.
        longitude (float): 입력 경도.
        top_k (int): 추천할 응급실 개수. 기본값은 3.

    Returns:
        pd.DataFrame: 추천된 응급실 정보 데이터프레임 (없으면 None).
    """
    path = "/content/drive/MyDrive/project6_2/"
    csv_path = path + "응급실 정보.csv"
    model_directory = path + "fine_tuned_bert"

    try:
        # 파일명 추출
        filename = os.path.basename(audio_path)

        # 파이프라인 초기화
        pipeline = EmergencyPipeline(
            path=path,
            csv_path=csv_path,
            model_directory=model_directory
        )

        # 파이프라인 실행
        recommended_rooms = pipeline.run(
            audio_path=os.path.dirname(audio_path),
            filename=filename,
            input_coords=(latitude, longitude),
            top_k=top_k
        )

        if recommended_rooms is not None:
            # 인덱스 초기화
            recommended_rooms.reset_index(drop=True, inplace=True)

            # 필요한 컬럼만 선택 및 'Distance' 칼럼 이름 수정
            result = recommended_rooms[['병원이름', '주소', '전화번호 1', '전화번호 3', 'Distance']].rename(columns={'Distance': '거리'})

            return result

    except Exception as e:
        print(f"파이프라인 실행 중 오류 발생: {e}")
        return None


In [62]:
audio_path = "/content/drive/MyDrive/project6_2/audio2.mp3"
latitude = 37.3586184462591
longitude = 127.1150358982930

# 함수 실행
result = run_emergency_pipeline(audio_path, latitude, longitude)

[Step 1] 오디오를 텍스트로 변환 중...
Transcript: 119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 아프고 좀 띵한 것 같아요. 우한이 좀 들어요. 어떻게 해야 할까요?
[Step 2] 텍스트를 요약 중...
Summary: 열이 38도로 상승하여 머리가 아프고 띵한 증상이 나타나는 중등도 열성 질환 환자. 의사와 상담하여 약물 치료 및 안정을 유지하는 것이 필요합니다.
[Step 3] 텍스트 등급 예측 중...
Predicted Grade: 3
집에서 요양하면 충분할듯합니다.
